# Imports and Setups

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import plotnine as p9
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
import transformer
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from fairnesTester import FairnessTester
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from fairnesTester import FairnessTester

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from xgboost import XGBClassifier

#pipelines and transformer

cat_trans = Pipeline(steps=[
    ("selector", transformer.DataSelector("object")),
    ("one_hot", preprocessing.OneHotEncoder())
])
num_trans = Pipeline(steps=[
    ("selector", transformer.DataSelector("number")),
    ("scaler", StandardScaler() )
])

pre_pipe = FeatureUnion(transformer_list=[
    ("cat", cat_trans),
    ("num", num_trans)
])

lb = LabelBinarizer()
del_nan = transformer.DeleteNAN("")

# Importing and preparing the data


## Adult Income Data set

In [99]:
filename = "Datasets/adult.data"
names = ["age", "workclass", "fnlwgt","education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "class"]
train = pd.read_csv(filename, names=names)
test = test = pd.read_csv("Datasets/adult.test", names=names)

del_nan.set_nan_char(" ?")
train = del_nan.transform(train)
test = del_nan.transform(test)

train["class"] = lb.fit_transform(train["class"])
test["class"] = lb.fit_transform(test["class"])

train_data = pre_pipe.fit_transform(train.drop("class", axis=1))
train_labels = train["class"]

test_data = pre_pipe.transform(test.drop("class", axis=1))
test_labels = test["class"]

#attribute for Fairness tester
dataset_name = "Adult_Income"
priv_val = " Male"
unpriv_val = " Female"
protected_att = "sex"

## German Credit Dataset

In [110]:
filename = "Datasets/german.data"
names = ["status existing account","duration", "credit history", "purpose", "credit amount", "savings", "employment since", "installment rate", "sex", "other debtors", "residence since", "property", "age", "installment plans", "housing", "num existing credits", "job", "no of pople liable", "telephone", "foreign worker", "class" ]
data = pd.read_csv(filename, sep=" ", names =names)

data["class"] = lb.fit_transform(data["class"])

data, test, train_labels, test_labels = train_test_split(data, data["class"], random_state=42)

train_data = pre_pipe.fit_transform(data.drop("class", axis=1))
test_data = pre_pipe.transform(test.drop("class", axis=1))

#transform for Fairness tester
test["sex"].replace(["A91","A93", "A94"],"m",inplace=True)
test["sex"].replace(["A92","A95"],"f",inplace=True)

#attribute for Fairness tester
dataset_name = "German_Credit"
priv_val = "m"
unpriv_val = "f"
protected_att = "sex"


## Default of Creddit Card Payments

In [121]:
filename = "Datasets/default of credit.xls"
data_inp = pd.read_excel(filename, dtype={"X1": int,"X2": object,"X3": object,"X4": object,"X5": object,"X6": object,"X7": object,"X8": object,"X9": object,"X10": object,"X11": object,"X12": int,"X13": int,"X14": int,"X15": int,"X16": int,"X17": int,"X23": int,"X18": int,"X19": int,"X20": int,"X21": int,"X22": int})

data_inp = data_inp.rename(columns={"Y": "class"})

data, test, train_labels, test_labels = train_test_split(data_inp, data_inp["class"], random_state=42)

pre_pipe.fit(data_inp.drop("class", axis=1))
train_data = pre_pipe.transform(data.drop("class", axis=1))
test_data = pre_pipe.transform(test.drop("class", axis=1))


#attribute for Fairness tester
dataset_name = "default_of_credit"
priv_val = 1 #male
unpriv_val = 2 #female
protected_att = "X2"



## Rici vs Stefano Dataset

In [ ]:
filename = "Datasets/ricci.csv"
data_inp = pd.read_csv(filename).drop("Unnamed: 0", axis=1)
#applicants with combine >= 70 pass
#read paper Did the Results of Promotion Exams Have a Disparate Impact on Minorities? Using Statistical Evidence in Ricci v. DeStefano
data_inp.rename(columns={"Combine": "class"}, inplace=True)

data_inp.loc[(data_inp["class"]<70), "class"] = 0
data_inp.loc[data_inp["class"]>=70, "class"] = 1


data, test, train_labels, test_labels = train_test_split(data_inp, data_inp["class"], random_state=42)

pre_pipe.fit(data_inp.drop("class", axis=1))
train_data = pre_pipe.transform(data.drop("class", axis=1))
test_data = pre_pipe.transform(test.drop("class", axis=1))

#transform for Fairness tester
test["Race"].replace(["H","B",],"NW",inplace=True)


#attribute for Fairness tester
dataset_name = "ricci_vs_stefano"
priv_val = "W" #white
unpriv_val = "NW" #not white
protected_att = "Race"


## Heart Disease Dataset

In [ ]:
filename = "Datasets/processed.cleveland.data"
names = ["age", "sex", 3,4,5,6,7,8,9,10,11,12,13,"class"]
data_inp = pd.read_csv(filename, names=names)


data_inp.loc[data_inp["class"]>=1, "class"] = 1 #existing heart disase

data, test, train_labels, test_labels = train_test_split(data_inp, data_inp["class"], random_state=42)

pre_pipe.fit(data_inp.drop("class", axis=1))
train_data = pre_pipe.transform(data.drop("class", axis=1))
test_data = pre_pipe.transform(test.drop("class", axis=1))


#attribute for Fairness tester
dataset_name = "heart_diseases_dataset"
priv_val = 1 #male
unpriv_val = 0 #female
protected_att = "sex"

## Heart Failure Dataset

In [ ]:
filename = "Datasets/heart_failure.csv"
data_inp = pd.read_csv(filename)
data_inp.rename(columns={"DEATH_EVENT":"class"}, inplace=True)

data, test, train_labels, test_labels = train_test_split(data_inp, data_inp["class"], random_state=42)


pre_pipe.fit(data_inp.drop("class", axis=1))
train_data = pre_pipe.transform(data.drop("class", axis=1))
test_data = pre_pipe.transform(test.drop("class", axis=1))


#attribute for Fairness tester
dataset_name = "heart_failure_dataset"
priv_val = 1 #male
unpriv_val = 0 #female
protected_att = "sex"

## Student Performance Data Set

In [ ]:
filename = "Datasets/student-por.csv"
data_inp = pd.read_csv(filename, sep=";")

data_inp.drop(["G1","G2"],axis=1,inplace=True)
data_inp.rename(columns={"G3":"class"},inplace=True)

data_inp.loc[(data_inp["class"]<10), "class"] = 0 #failed
data_inp.loc[data_inp["class"]>=10, "class"] = 1 #passed


data, test, train_labels, test_labels = train_test_split(data_inp, data_inp["class"], random_state=42)


pre_pipe.fit(data_inp.drop("class", axis=1))
train_data = pre_pipe.transform(data.drop("class", axis=1))
test_data = pre_pipe.transform(test.drop("class", axis=1))


#attribute for Fairness tester
dataset_name = "student_performance_dataset"
priv_val = "M" #male
unpriv_val = "F" #female
protected_att = "sex"



# Classifiers

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(train_data,train_labels)

tree_pred = tree.predict(test_data)
test["prediction"] = tree_pred
dataset_name

## Random Forest

In [ ]:
forest = RandomForestClassifier()
forest.fit(train_data,train_labels)

forest_pred = forest.predict(test_data)
test["prediction"] = forest_pred
dataset_name

In [ ]:
features = ["X", "Race_White", "Race_Black", "Race_Hispanic", "Position_Leutantent", "Position_Captain", "oral", "written"]
export_graphviz(tree,out_file="decision_tree_graph" , feature_names= features, fill=True)

## K-Nearest-Neighbour

In [ ]:
knn = KNeighborsClassifier(3)
knn.fit(train_data,train_labels)

knn_pred = knn.predict(test_data)
test["prediction"] = knn_pred
dataset_name 

## Support Vector Machine

In [ ]:
#nochmal dringend anschauen
svc = SVC(kernel="rbf", C=0.025, probability=True) 
svc.fit(train_data,train_labels)

svc_pred = svc.predict(test_data)
test["prediction"] = svc_pred
dataset_name

## Ada Boost 

In [ ]:
ada = AdaBoostClassifier()
svc.fit(train_data,train_labels)

svc_pred = svc.predict(test_data)
test["prediction"] = svc_pred
dataset_name

## Gaussian NB

In [ ]:
gnb = GaussianNB()
gnb.fit(train_data.toarray(),train_labels)

gnb_pred = gnb.predict(test_data.toarray())
test["prediction"] = gnb_pred
dataset_name

## Gaussian Proccess Classifier

In [ ]:
gpc = GaussianProcessClassifier()
gpc.fit(train_data.toarray(),train_labels)

gpc_pred = gpc.predict(test_data.toarray())
test["prediction"] = gpc_pred
gpc.__class__.__name__

## XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(train_data.toarray(),train_labels)

xgb_pred = xgb.predict(test_data.toarray())
test["prediction"] = xgb_pred

# Classifier List


In [122]:
classifiers = [DecisionTreeClassifier(),RandomForestClassifier(),SVC(),AdaBoostClassifier(),KNeighborsClassifier(5), GaussianNB(), XGBClassifier()]
model_names = []

for model in classifiers:
    
    name = model.__class__.__name__
    print(name)
    model_names.append(name)

    model.fit(train_data.toarray(), train_labels)
    pred = model.predict(test_data.toarray())

    test[name]=pred

DecisionTreeClassifier


/tmp/ipykernel_3037/4127379242.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


RandomForestClassifier


/tmp/ipykernel_3037/4127379242.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SVC


...

AdaBoostClassifier


/tmp/ipykernel_3037/4127379242.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KNeighborsClassifier


/tmp/ipykernel_3037/4127379242.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_3037/4127379242.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/maxk/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers star

GaussianNB
XGBClassifier
[18:09:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Testing for Fairness


## testing one model

In [ ]:
tester = FairnessTester()
tester.setup(test, protected_att, priv_val, unpriv_val)
print("privileged confusion: \n",tester.priv_confusion_matrix())
print("unprivileged confusion: \n", tester.unpriv_confusion_matrix())
tester.confuison_based_dic()

## testing classifiers list

In [123]:
tester = FairnessTester()


result_df = pd.DataFrame()

for name in model_names:
    tester.setup(test, protected_att, priv_val, unpriv_val, name)
    result_dic = {"model": name}
    result_dic.update(tester.confusion_based_dic_priv())
    result_df= result_df.append(result_dic, ignore_index=True)
    
    result_dic = {"model": name}
    result_dic.update(tester.confusion_based_dic_unpriv())
    result_df= result_df.append(result_dic, ignore_index=True)
definitions_names = list(tester.confuison_based_dic().keys())
result_df
    

,model,group,statistical parity,predictive parity,negative predictive parity,equal opportunity,predictive equality,overall accuracy equality,treatment equality
0,DecisionTreeClassifier,priv,0.233169,0.394366,0.815418,0.393812,0.184233,0.717241,0.303920
1,DecisionTreeClassifier,unpriv,0.225365,0.367530,0.841495,0.402838,0.179429,0.734680,0.300470
2,RandomForestClassifier,priv,0.129392,0.624365,0.824594,0.345992,0.063410,0.798686,0.096957
3,RandomForestClassifier,unpriv,0.112907,0.656064,0.851721,0.360262,0.048884,0.829630,0.076412
4,SVC,priv,0.094253,0.686411,0.813633,0.277075,0.038560,0.801642,0.053339
5,SVC,unpriv,0.096521,0.686047,0.845714,0.322052,0.038146,0.830303,0.056267
6,AdaBoostClassifier,priv,0.114943,0.665714,0.822635,0.327707,0.050129,0.804598,0.074564
7,AdaBoostClassifier,unpriv,0.096296,0.673660,0.844262,0.315502,0.039559,0.827834,0.057793
8,KNeighborsClassifier,priv,0.133662,0.520885,0.810842,0.298172,0.083548,0.772085,0.119043
9,KNeighborsClassifier,unpriv,0.116723,0.580769,0.843964,0.329694,0.061599,0.813244,0.091897


In [124]:
#fit all data into better selectable format(for complete plot of all definitions)

full_result_df = pd.DataFrame()
for model in model_names:
    for defi in definitions_names:
        for group in ["priv", "unpriv"]:    
            #if defi == "treatment equality":   #skip treatment equality as it will destroy the scales for full plot
            #    continue          
            dic = {}
            dic["model"] = result_df.loc[(result_df["model"]==model)&(result_df["group"]==group)]["model"].item()
            dic["group"] = result_df.loc[(result_df["model"]==model)&(result_df["group"]==group)]["group"].item()
            dic["definition"] = defi
            dic["result"]= result_df.loc[(result_df["model"]==model)&(result_df["group"]==group)][defi].item()
            full_result_df = full_result_df.append(dic, ignore_index=True)

full_result_df
        

,model,group,definition,result
0,DecisionTreeClassifier,priv,statistical parity,0.233169
1,DecisionTreeClassifier,unpriv,statistical parity,0.225365
2,DecisionTreeClassifier,priv,predictive parity,0.394366
3,DecisionTreeClassifier,unpriv,predictive parity,0.367530
4,DecisionTreeClassifier,priv,negative predictive parity,0.815418
...,...,...,...,...
93,XGBClassifier,unpriv,predictive equality,0.055100
94,XGBClassifier,priv,overall accuracy equality,0.800328
95,XGBClassifier,unpriv,overall accuracy equality,0.824691
96,XGBClassifier,priv,treatment equality,0.105137


In [125]:
# get differences from data

differences = pd.DataFrame()

for model in model_names:
    for defi in definitions_names:   
        if defi == "treatment equality":   #skip treatment equality as it will destroy the scales for full plot
            continue          
        dic = {}
        dic["model"] = result_df.loc[(result_df["model"]==model)&(result_df["group"]==group)]["model"].item()
        x = result_df.loc[(result_df["model"]==model)&(result_df["group"]=="priv")][defi].item()
        y = result_df.loc[(result_df["model"]==model)&(result_df["group"]=="unpriv")][defi].item()
        dic["definition"] = defi
        dic["difference"]= abs(x-y)
        differences = differences.append(dic, ignore_index=True)

In [126]:
# get ratio from data

ratio = pd.DataFrame()

for model in model_names:
    for defi in definitions_names:   
        if defi == "treatment equality":   #skip treatment equality as it will destroy the scales for full plot
            continue          
        dic = {}
        dic["model"] = result_df.loc[(result_df["model"]==model)&(result_df["group"]==group)]["model"].item()
        x = result_df.loc[(result_df["model"]==model)&(result_df["group"]=="priv")][defi].item()
        y = result_df.loc[(result_df["model"]==model)&(result_df["group"]=="unpriv")][defi].item()
        dic["definition"] = defi
        if (x == 0):
            dic["ratio"]= 0
        else:
            dic["ratio"]= y/x
        
        ratio = ratio.append(dic, ignore_index=True)

# Plotting

In [127]:
#one plot for each definition with color
plots = []
for definition in definitions_names:

    plot = (p9.ggplot(data= result_df, mapping = p9.aes(x="model", y=definition, fill="group")) 
    + p9.geom_col(position="dodge")
    + p9.theme(axis_text_x = p9.element_text(angle=90))
    + p9.labs(x="ML Models", y= "Results", title=(definition+" for " + dataset_name)))
    plots.append(plot) 

i=0
for plot in plots:
    plot.save(filename="plots/"+dataset_name+"/"+dataset_name+"_"+definitions_names[i]+"_color.png")
    i+=1

/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_statistical parity_color.png
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_predictive parity_color.png
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_negative predictive parity_color.png
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/maxk/.local/lib/python3.8/s

In [128]:
#overall plot for all definitions with color
#skip treatment equality as it will destory the scales
plot = (p9.ggplot(data= full_result_df.drop(full_result_df[full_result_df.definition=="treatment equality"].index), mapping = p9.aes(x="model", y="result", fill="group")) 
    + p9.geom_col(position="dodge")
    + p9.facet_grid(".~definition") 
    + p9.theme(axis_text_x = p9.element_text(angle=90))
    + p9.labs(x="ML Models", y= "Results", title=(" Complete results for " + dataset_name))
    )
plot.save(filename="plots/"+dataset_name+"/"+dataset_name+"_complete_color.png", height=4 , width = 17)

/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 17 x 4 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_complete_color.png


In [129]:
# plot differences
plot = (p9.ggplot(data= differences, mapping = p9.aes(x="model", y="difference")) 
    + p9.geom_col()
    + p9.facet_grid(".~definition", space="free_x", scales="fixed") 
    + p9.theme(axis_text_x = p9.element_text(angle=90))
    + p9.labs(x="ML Models", y= "Differences", title=("Differences for " + dataset_name))
    + p9.ylim(0,1)
    )
plot.save(filename="plots/"+dataset_name+"/"+dataset_name+"_differences_scaled.png", height=4 , width = 17)


/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 17 x 4 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_differences_scaled.png


In [130]:
# plot differences
plot = (p9.ggplot(data= differences, mapping = p9.aes(x="model", y="difference")) 
    + p9.geom_col()
    + p9.facet_grid(".~definition", space="free_x", scales="fixed") 
    + p9.theme(axis_text_x = p9.element_text(angle=90))
    + p9.labs(x="ML Models", y= "Differences", title=("Differences for " + dataset_name))
    )
plot.save(filename="plots/"+dataset_name+"/"+dataset_name+"_differences_close.png", height=4 , width = 17)

/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 17 x 4 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_differences_close.png


In [131]:
#plot ratio
plot = (p9.ggplot(data= ratio, mapping = p9.aes(x="model", y="ratio")) 
    + p9.geom_col()
    + p9.facet_grid(".~definition", space="free_x", scales="free") 
    + p9.theme(axis_text_x = p9.element_text(angle=90))
    + p9.labs(x="ML Models", y= "Ratio", title=("Ratio for " + dataset_name))
    )
plot.save(filename="plots/"+dataset_name+"/"+dataset_name+"_ratio.png", height=4 , width = 17)

/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 17 x 4 in image.
/home/maxk/.local/lib/python3.8/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: plots/default_of_credit/default_of_credit_ratio.png
